## SBC (single binary classifier) reduction and classification

In [3]:
import numpy as np
import pandas as pd

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
X = pd.DataFrame(X)
y = np.array([0, 1, 2, 3])
y = pd.DataFrame(y)

- for K classes
- find K-1 parallel hyperplanes.
- s is the number of classes to the left / right of the hyperplane.
- for s = K-1,
- an hyperplane separates 2 sets of classes.
- each hyperplane is a binary classification problem, with the additional constraint of parallelism.
- all these problems have to be solved simultaneously in an augmented feature space
    - for x in R^d, the augmented feature space is R^(d+1)
- for each x create (K-1) new points: (x, 0)(x,h1)...(x,hK-2), where h is a positive constant
- define a binary training set
    - (x1,0) belongs to C1, (x2,0)...(xK,0) belong to C2
    - (x1,h1) (x2,h1) belongs to C1, (x3,h1)...(xK, h1) belong to C2
    - ...
    - (x1,hK-2)...(xK-1,hK-2) belong to C1, (xK,hK-2) belongs to C2

In [4]:
def sbc_reduction(X, y, h=1):
    # num of classes
    K = len(np.unique(y))
    
    # num of parallel hyperplanes to be created (and replicas)
    s = K-1
    
    # for each point, create s replicas each with a new feature in [0, h, h*2, ... h*(s-1)]
    # the new label is a binary label
    new_X = []
    new_y = []
    for i in range(X.shape[0]): # for each point
        for j in range(s): # for each replica
            new_X.append(np.append(X.iloc[i].values, h*j))
            new_label = y.iloc[i] <= j
            new_y.append(new_label.astype(int))
    
    new_X = pd.DataFrame(new_X).reset_index(drop=True)
    new_y = pd.DataFrame(new_y).reset_index(drop=True)
    new_data = pd.concat([new_X, new_y], axis=1)
    
    return new_X, new_y, new_data


new_X, new_y, new_data = sbc_reduction(X, y)
print(new_data)

    0  1  2  0
0   1  2  0  1
1   1  2  1  1
2   1  2  2  1
3   3  4  0  0
4   3  4  1  1
5   3  4  2  1
6   5  6  0  0
7   5  6  1  0
8   5  6  2  1
9   7  8  0  0
10  7  8  1  0
11  7  8  2  0


apply a linear two class classifier

In [5]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.solver = 'liblinear'
logistic.penalty = 'l1'
logistic.fit(new_X, new_y.values.ravel())  # convert y to 1d array
pred_sbc_y = logistic.predict(new_X)
print(pred_sbc_y)

[0 1 1 0 0 1 0 0 1 0 0 0]


to classify an example
- classify all its replicas
- get a sequence of K-1 labels
- from this sequence, infer the class of the original example following the rule:
    (for K=3)
    - if both are C1, then the class is C1
    - if one is C1 and the other is C2, then the class is C2
    - if both are C2, then the class is C3

In [6]:
def sbc_classif(og_X, og_y, sbc_X, pred_sbc_y):
    K = len(np.unique(og_y)) # num of classes
    s = K-1 # num of hyperplanes / replicas
    
    # get classification of all replicas of each point
    all_labels = [pred_sbc_y[i:i + s] for i in range(0, len(pred_sbc_y), s)]
    all_labels = np.array(all_labels)
    print(all_labels)
    
    # get the class of the point
    # if all replicas are 0, then the class is 0
    # if one is 1 and the rest are 0, then the class is 1
    # if two are 1 and the rest are 0, then the class is 2
    # ...
    # if all replicas are 1, then the class is K
    final_labels = np.argmax(all_labels, axis=1)    
    
    return final_labels


final_labels = sbc_classif(X, y, new_X, pred_sbc_y)
print(final_labels)

[[0 1 1]
 [0 0 1]
 [0 0 1]
 [0 0 0]]
[1 2 2 0]
